# Clustering Crypto

In [18]:
# Initial imports
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [42]:
# Load the crypto_data.csv dataset.
# YOUR CODE HERE
file_path ="crypto_data.csv"
crypto_df = pd.read_csv(file_path,index_col=0)
crypto_df["TotalCoinSupply"] = pd.to_numeric(crypto_df["TotalCoinSupply"].str.replace(" ",""), errors='coerce')
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,4.200000e+01
365,365Coin,X11,True,PoW/PoS,NaN,2.300000e+09
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,5.320000e+08
611,SixEleven,SHA-256,True,PoW,NaN,6.110000e+05
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0.000000e+00
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,3.141593e+11
2015,2015 coin,X11,True,PoW/PoS,NaN,0.000000e+00
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,2.100000e+07
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0.000000e+00
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,8.400000e+07


In [43]:
crypto_df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
dtype: object

In [44]:
# Keep all the cryptocurrencies that are being traded.
# YOUR CODE HERE
crypto_df = crypto_df[crypto_df["IsTrading"]==True]
print(crypto_df.shape)

(1144, 6)


In [45]:
# Keep all the cryptocurrencies that have a working algorithm.
# YOUR CODE HERE

In [46]:
# Remove the "IsTrading" column. 
# YOUR CODE HERE
crypto_df_useful_columns= crypto_df.drop(columns=["IsTrading"])
print(crypto_df_useful_columns.shape)
crypto_df.head(5)

(1144, 5)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,4.200000e+01
365,365Coin,X11,True,PoW/PoS,NaN,2.300000e+09
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,5.320000e+08
611,SixEleven,SHA-256,True,PoW,NaN,6.110000e+05
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0.000000e+00


In [47]:
# Remove rows that have at least 1 null value.
# YOUR CODE HERE
crypto_df_nona = crypto_df_useful_columns.dropna(axis=0)
print(crypto_df_nona.shape)
crypto_df_nona.head(10)

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
808,808,SHA-256,PoW/PoS,0.000000e+00,0.000000e+00
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.141593e+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07
ETH,Ethereum,Ethash,PoW,1.076842e+08,0.000000e+00
LTC,Litecoin,Scrypt,PoW,6.303924e+07,8.400000e+07
DASH,Dash,X11,PoW/PoS,9.031294e+06,2.200000e+07
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,2.100000e+08


In [57]:
# Keep the rows where coins are mined.
# YOUR CODE HERE
crypto_df_mined = crypto_df_nona[crypto_df_nona["TotalCoinsMined"]>0]
print(crypto_df_mined.shape)
crypto_df_mined.head()

(532, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.141593e+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07
ETH,Ethereum,Ethash,PoW,1.076842e+08,0.000000e+00


In [65]:
# Create a new DataFrame that holds only the cryptocurrencies names.
# YOUR CODE HERE
crypto_df_name = crypto_df_mined[["CoinName"]]
print(crypto_df_name.shape)
crypto_df_name.head(10)

(532, 1)


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
LTC,Litecoin
DASH,Dash
XMR,Monero
ETC,Ethereum Classic
ZEC,ZCash


In [50]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
# YOUR CODE HERE
crypto_df_dropname = crypto_df_mined.drop(columns="CoinName")
print(crypto_df_dropname.shape)
crypto_df_dropname.head()

(532, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07
ETH,Ethash,PoW,1.076842e+08,0.000000e+00


In [51]:
# Use get_dummies() to create variables for text features.
# YOUR CODE HERE
crypto_df_encoded = pd.get_dummies(crypto_df_dropname)
print(crypto_df_encoded.shape)
crypto_df_encoded.head(5)

(532, 98)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
# Standardize the data with StandardScaler().
# YOUR CODE HERE
crypto_scaled = StandardScaler().fit_transform(crypto_df_encoded)
crypto_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [61]:
# Using PCA to reduce dimension to three principal components.
# YOUR CODE HERE
pca = PCA(n_components=3)
crypto_scaled_pca = pca.fit_transform(crypto_scaled)
crypto_scaled_pca


array([[-0.33152945,  1.0497867 , -0.57088359],
       [-0.31484136,  1.04998088, -0.57133982],
       [ 2.31956805,  1.72683982, -0.58965159],
       ...,
       [ 0.32368301, -2.31247811,  0.39122431],
       [-0.12476653, -2.03772509,  0.45223495],
       [-0.30037206,  0.8014847 , -0.27196454]])

In [68]:
# Create a DataFrame with the three principal components.
# YOUR CODE HERE
pcs_df  = pd.DataFrame(data=crypto_scaled_pca,columns =["pc1","pc2","pc3"],index =crypto_df_name.index )
print(pcs_df .shape)
pcs_df .head(10)

(532, 3)


,pc1,pc2,pc3
42,-0.331529,1.049787,-0.570884
404,-0.314841,1.049981,-0.571340
1337,2.319568,1.726840,-0.589652
BTC,-0.149174,-1.344274,0.162256
ETH,-0.154143,-2.018148,0.372984
LTC,-0.167258,-1.126902,-0.032749
DASH,-0.377841,1.244527,-0.557113
XMR,-0.136748,-2.139536,0.460281
ETC,-0.152584,-2.018244,0.372961
ZEC,-0.124765,-2.037725,0.452235


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [70]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE
inertia = []
k = list(range(1, 10))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [71]:
# Initialize the K-Means model.
# YOUR CODE HERE
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
# YOUR CODE HERE
model.fit(pcs_df)
# Predict clusters
# YOUR CODE HERE
predictions = model.predict(pcs_df)
predictions

array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,

In [75]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# YOUR CODE HERE
clustered_df = pd.concat([crypto_df_dropname,pcs_df],axis=1,)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE
clustered_df["CoinName"] = crypto_df_name
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE
clustered_df["Class"] = predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,pc1,pc2,pc3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.331529,1.049787,-0.570884,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.314841,1.049981,-0.571340,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.319568,1.726840,-0.589652,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.149174,-1.344274,0.162256,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.154143,-2.018148,0.372984,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.167258,-1.126902,-0.032749,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.377841,1.244527,-0.557113,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.136748,-2.139536,0.460281,Monero,1
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.152584,-2.018244,0.372961,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.124765,-2.037725,0.452235,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [93]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE
px.scatter_3d(clustered_df,x="pc1",y="pc2",z="pc3",color="Class",hover_name="CoinName",hover_data=['Algorithm'])

In [95]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE
clustered_df.hvplot.table(columns=['CoinName','Algorithm','ProofType','TotalCoinSupply','TotalCoinsMined','Class'],sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [99]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE
print(f"There are {len(clustered_df )} tradable cryptocurrencies")

There are 532 tradable cryptocurrencies


In [114]:
clustered_df.loc[:, ['TotalCoinSupply', 'TotalCoinsMined']]


,TotalCoinSupply,TotalCoinsMined
42,4.200000e+01,4.199995e+01
404,5.320000e+08,1.055185e+09
1337,3.141593e+11,2.927942e+10
BTC,2.100000e+07,1.792718e+07
ETH,0.000000e+00,1.076842e+08
...,...,...
ZEPH,2.000000e+09,2.000000e+09
GAP,2.500000e+08,1.493105e+07
BDX,1.400223e+09,9.802226e+08
ZEN,2.100000e+07,7.296538e+06


In [115]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE
scaler = MinMaxScaler()
scaled = scaler.fit_transform(clustered_df.loc[:, ['TotalCoinSupply', 'TotalCoinsMined']])
print(scaled)

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 ...
 [1.40022261e-03 9.90135079e-04]
 [2.10000000e-05 7.37028150e-06]
 [1.00000000e-06 1.29582282e-07]]


In [119]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE
plot_df = pd.DataFrame(scaled,columns=['TotalCoinSupply','TotalCoinsMined'], index =crypto_df_name.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE
plot_df["CoinName"] = clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE
plot_df["Class"] = clustered_df["Class"]

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [122]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply",by="Class",hover_cols=['CoinName'] )

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)